In [1]:
import tkinter as tk
from tkinter import messagebox
import random

class Forca:
    def __init__(self, master):
        self.master = master
        master.title("Jogo da Forca")

        # Lista de palavras em português
        self.words = ["python", "programacao", "desenvolvimento", "algoritmo", "interface", "computador"]
        self.word = self.get_random_word().upper()
        self.guessed_letters = set()
        self.attempts_left = 6  # Número de tentativas permitidas
        self.create_widgets()
        self.update_display()

    def get_random_word(self):
        """Retorna uma palavra aleatória da lista de palavras."""
        return random.choice(self.words)

    def create_widgets(self):
        # Área de exibição da palavra com letras adivinhadas
        self.word_label = tk.Label(self.master, font=("Arial", 24))
        self.word_label.pack(pady=20)

        # Área de entrada para adivinhar letras
        self.guess_entry = tk.Entry(self.master, font=("Arial", 18))
        self.guess_entry.pack(pady=10)

        # Botão para adivinhar
        self.guess_button = tk.Button(self.master, text="Adivinhar", command=self.make_guess, font=("Arial", 14))
        self.guess_button.pack(pady=10)

        # Botão para reiniciar o jogo
        self.restart_button = tk.Button(self.master, text="Reiniciar Jogo", command=self.restart_game, font=("Arial", 14))
        self.restart_button.pack(pady=10)

        # Área para mostrar o número de tentativas restantes
        self.attempts_label = tk.Label(self.master, font=("Arial", 14))
        self.attempts_label.pack(pady=10)

    def update_display(self):
        """Atualiza a exibição da palavra e o número de tentativas restantes."""
        display_word = ' '.join(letter if letter in self.guessed_letters else '_' for letter in self.word)
        self.word_label.config(text=display_word)
        self.attempts_label.config(text=f"Tentativas restantes: {self.attempts_left}")

        if '_' not in display_word:
            messagebox.showinfo("Parabéns!", "Você ganhou!")
            self.master.quit()
        elif self.attempts_left <= 0:
            messagebox.showinfo("Game Over", f"Você perdeu! A palavra era: {self.word}")
            self.master.quit()

    def make_guess(self):
        """Faz uma tentativa de adivinhar uma letra."""
        guess = self.guess_entry.get().upper()
        self.guess_entry.delete(0, tk.END)

        if len(guess) != 1 or not guess.isalpha():
            messagebox.showwarning("Aviso", "Por favor, insira uma única letra.")
            return

        if guess in self.guessed_letters:
            messagebox.showinfo("Aviso", "Você já adivinhou essa letra.")
            return

        self.guessed_letters.add(guess)

        if guess not in self.word:
            self.attempts_left -= 1

        self.update_display()

    def restart_game(self):
        """Reinicia o jogo com uma nova palavra e contagem de tentativas."""
        self.word = self.get_random_word().upper()
        self.guessed_letters = set()
        self.attempts_left = 6
        self.update_display()

# Inicializa a interface gráfica
root = tk.Tk()
game = Forca(root)
root.mainloop()


In [4]:
import tkinter as tk
from tkinter import messagebox
import random

class Battleship:
    def __init__(self, master):
        self.master = master
        master.title("Batalha Naval")

        # Tamanho do grid e tamanhos dos navios
        self.grid_size = 10
        self.ship_sizes = [5, 4, 3, 3, 2]
        
        # Inicialização dos tabuleiros do jogador e do computador
        self.player_board = [[' ' for _ in range(self.grid_size)] for _ in range(self.grid_size)]
        self.computer_board = [[' ' for _ in range(self.grid_size)] for _ in range(self.grid_size)]
        self.computer_guesses = [[' ' for _ in range(self.grid_size)] for _ in range(self.grid_size)]
        
        # Flags de controle do jogo
        self.ships_placed = False  # Verifica se os navios foram colocados
        self.game_over = False  # Verifica se o jogo terminou
        self.hunting_mode = False  # Modo de busca do computador quando encontra um navio
        self.last_hit = None  # Coordenadas do último acerto do computador

        self.create_widgets()  # Cria os elementos da interface gráfica
        self.place_ships_randomly(self.computer_board)  # Posiciona os navios do computador aleatoriamente

    def create_widgets(self):
        # Criação dos botões para o tabuleiro do jogador
        self.player_buttons = []
        for i in range(self.grid_size):
            row = []
            for j in range(self.grid_size):
                button = tk.Button(self.master, text=" ", width=3, height=2, 
                                   command=lambda i=i, j=j: self.player_button_click(i, j))
                button.grid(row=i, column=j)
                row.append(button)
            self.player_buttons.append(row)

        # Criação dos botões para o tabuleiro do computador
        self.computer_buttons = []
        for i in range(self.grid_size):
            row = []
            for j in range(self.grid_size):
                button = tk.Button(self.master, text=" ", width=3, height=2, 
                                   command=lambda i=i, j=j: self.computer_button_click(i, j))
                button.grid(row=i, column=j + self.grid_size + 2)
                row.append(button)
            self.computer_buttons.append(row)

        # Criação do botão para reiniciar o jogo
        restart_button = tk.Button(self.master, text="Reiniciar Jogo", command=self.restart_game)
        restart_button.grid(row=self.grid_size, column=0, columnspan=self.grid_size, pady=10)

    def place_ships_randomly(self, board):
        # Método para posicionar os navios de forma aleatória no tabuleiro
        for ship_size in self.ship_sizes:
            placed = False
            while not placed:
                orientation = random.choice(['H', 'V'])
                if orientation == 'H':
                    row = random.randint(0, self.grid_size - 1)
                    col = random.randint(0, self.grid_size - ship_size)
                    if all(board[row][col + i] == ' ' for i in range(ship_size)):
                        for i in range(ship_size):
                            board[row][col + i] = 'S'
                        placed = True
                else:
                    row = random.randint(0, self.grid_size - ship_size)
                    col = random.randint(0, self.grid_size - 1)
                    if all(board[row + i][col] == ' ' for i in range(ship_size)):
                        for i in range(ship_size):
                            board[row + i][col] = 'S'
                        placed = True

    def player_button_click(self, i, j):
        # Clique do jogador no seu tabuleiro para posicionar navios ou atacar
        if not self.ships_placed:
            self.place_player_ship(i, j)
        else:
            messagebox.showinfo("Informação", "O jogo já começou. Ataque o tabuleiro do oponente.")

    def place_player_ship(self, i, j):
        # Método para posicionar os navios do jogador
        # Este é um exemplo simples para fins de demonstração.
        if self.player_board[i][j] == ' ':
            self.player_board[i][j] = 'S'
            self.player_buttons[i][j].config(text="S", bg="gray")
            # Verifica se todos os navios foram posicionados
            if all(cell == 'S' for row in self.player_board for cell in row):
                self.ships_placed = True  # Todos os navios foram posicionados
                messagebox.showinfo("Informação", "Todos os navios foram posicionados. O jogo pode começar.")

    def computer_button_click(self, i, j):
        # Clique do jogador no tabuleiro do computador para atacar
        if self.game_over:
            return

        if self.computer_board[i][j] == 'S':  # Se o ataque acertou um navio
            self.computer_board[i][j] = 'X'
            self.computer_buttons[i][j].config(text="X", bg="red")
            self.check_winner()  # Verifica se alguém venceu
        else:  # Se o ataque errou
            self.computer_board[i][j] = 'O'
            self.computer_buttons[i][j].config(text="O", bg="blue")
        
        self.computer_turn()  # Turno do computador

    def computer_turn(self):
        # Turno do computador para atacar o jogador
        if self.game_over:
            return

        if self.hunting_mode and self.last_hit:  # Se o computador está em modo de busca
            i, j = self.get_adjacent_guess(*self.last_hit)
        else:  # Se o computador faz um ataque aleatório
            i, j = random.randint(0, self.grid_size - 1), random.randint(0, self.grid_size - 1)
            while self.computer_guesses[i][j] != ' ':
                i, j = random.randint(0, self.grid_size - 1), random.randint(0, self.grid_size - 1)

        self.computer_guesses[i][j] = 'G'
        if self.player_board[i][j] == 'S':  # Se o ataque do computador acertou
            self.player_board[i][j] = 'X'
            self.player_buttons[i][j].config(text="X", bg="red")
            self.last_hit = (i, j)  # Guarda a posição do acerto
            self.hunting_mode = True  # Ativa o modo de busca
            self.check_winner()
        else:  # Se o ataque do computador errou
            self.player_board[i][j] = 'O'
            self.player_buttons[i][j].config(text="O", bg="blue")
            if self.hunting_mode:
                self.hunting_mode = False  # Sai do modo de busca

    def get_adjacent_guess(self, row, col):
        """Obtém uma adivinhação válida adjacente à dada linha e coluna."""
        adjacent = [(row - 1, col), (row + 1, col), (row, col - 1), (row, col + 1)]
        valid_guesses = [(i, j) for i, j in adjacent if 0 <= i < self.grid_size and 0 <= j < self.grid_size and self.computer_guesses[i][j] == ' ']
        return random.choice(valid_guesses) if valid_guesses else (random.randint(0, self.grid_size - 1), random.randint(0, self.grid_size - 1))

    def check_winner(self):
        # Verifica se há um vencedor
        player_ships = sum(row.count('S') for row in self.player_board)
        computer_ships = sum(row.count('S') for row in self.computer_board)

        if player_ships == 0:
            self.game_over = True
            messagebox.showinfo("Fim de Jogo", "Você perdeu!")
        elif computer_ships == 0:
            self.game_over = True
            messagebox.showinfo("Fim de Jogo", "Você ganhou!")

    def restart_game(self):
        # Reinicia o jogo
        self.master.destroy()
        root = tk.Tk()
        battleship = Battleship(root)
        root.mainloop()

# Inicializa a interface gráfica e o jogo
root = tk.Tk()
battleship = Battleship(root)
root.mainloop()


In [3]:
import tkinter as tk
from tkinter import messagebox

class JogoDaVelha:
    def __init__(self):
        self.janela = tk.Tk()
        self.janela.title("Jogo da Velha")

        self.turno = "X"
        self.tabuleiro = [[" " for _ in range(3)] for _ in range(3)]

        self.botoes = []
        for i in range(3):
            linha = []
            for j in range(3):
                botao = tk.Button(self.janela, text=" ", width=10, height=5,
                                  command=lambda i=i, j=j: self.fazer_jogada(i, j))
                botao.grid(row=i, column=j)
                linha.append(botao)
            self.botoes.append(linha)

        # Botão Fechar
        botao_fechar = tk.Button(self.janela, text="Fechar", width=10, command=self.janela.quit)
        botao_fechar.grid(row=3, column=0)

        # Botão Reiniciar
        botao_reiniciar = tk.Button(self.janela, text="Reiniciar", width=10, command=self.reiniciar_jogo)
        botao_reiniciar.grid(row=3, column=2)

    def fazer_jogada(self, i, j):
        if self.tabuleiro[i][j] == " ":
            self.tabuleiro[i][j] = self.turno
            self.botoes[i][j].config(text=self.turno)
            self.verificar_vencedor()
            self.turno = "O" if self.turno == "X" else "X"
        else:
            messagebox.showwarning("Jogada Inválida", "Essa posição já está ocupada.")

    def verificar_vencedor(self):
        # Verificar linhas
        for linha in self.tabuleiro:
            if linha[0] == linha[1] == linha[2] and linha[0] != " ":
                self.mostrar_vencedor(linha[0])
                return

        # Verificar colunas
        for col in range(3):
            if self.tabuleiro[0][col] == self.tabuleiro[1][col] == self.tabuleiro[2][col] and self.tabuleiro[0][col] != " ":
                self.mostrar_vencedor(self.tabuleiro[0][col])
                return

        # Verificar diagonais
        if (self.tabuleiro[0][0] == self.tabuleiro[1][1] == self.tabuleiro[2][2] or
            self.tabuleiro[0][2] == self.tabuleiro[1][1] == self.tabuleiro[2][0]) and self.tabuleiro[1][1] != " ":
            self.mostrar_vencedor(self.tabuleiro[1][1])
            return

        # Verificar empate
        if all(all(cell != " " for cell in row) for row in self.tabuleiro):
            self.mostrar_vencedor("Empate")

    def mostrar_vencedor(self, vencedor):
        if vencedor == "Empate":
            messagebox.showinfo("Fim de Jogo", "O jogo terminou em empate!")
        else:
            messagebox.showinfo("Fim de Jogo", f"O jogador {vencedor} venceu!")
        self.reiniciar_jogo()

    def reiniciar_jogo(self):
        self.turno = "X"
        self.tabuleiro = [[" " for _ in range(3)] for _ in range(3)]
        for linha in self.botoes:
            for botao in linha:
                botao.config(text=" ")

    def iniciar(self):
        self.janela.mainloop()

if __name__ == "__main__":
    jogo = JogoDaVelha()
    jogo.iniciar()
